# 参数化装饰器

**在装饰器上层创建一个装饰器工厂，用来接受参数，它返回装饰器**

In [34]:
registry = set()

def register(active=True):    # 装饰器工厂函数，接受装饰器参数
    def decorate(func):       # 真正的装饰器
        print('running register(active=%s) -> decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func           # 装饰器必须返回一个函数
    return decorate           # 装饰器工厂函数必须返回装饰器

@register(active=False)
def f1():
    print('running f1()')

@register()     # 装饰器工厂函数必须作为函数调用，即使不传入参数，因为它本身不是装饰器，它的调用返回才是真正的装饰器
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False) -> decorate(<function f1 at 0x00000152D1813E18>)
running register(active=True) -> decorate(<function f2 at 0x00000152D18130D0>)


In [35]:
registry    # 很奇怪的发现当把装饰 f2()单独放到一个cell里面， 那个cell执行多次，这里的 registry里面是有多个 f2，字面上看到的都是一样的，但是registry是set，说明它们本质上不是同一个f2，每当执行一次会在内存的不同地方出现不同的f2

{<function __main__.f2>}

**如果不使用 @ 句法，那就是常规函数的调用**

In [36]:
register()(f3)

running register(active=True) -> decorate(<function f3 at 0x00000152D18137B8>)


<function __main__.f3>

In [37]:
registry

{<function __main__.f2>, <function __main__.f3>}

In [39]:
register(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x00000152D18130D0>)


<function __main__.f2>

In [40]:
registry

{<function __main__.f3>}

**这样会更好的理解装饰器是怎么回事：**

In [47]:
def deco_fac(*args1):
    def deco(func):
        def new_func(*args2):
            print(*args2)
        return new_func
    return deco

In [48]:
@deco_fac(1, 2, 3)
def f(a, b): pass

In [51]:
f('a', 'b')

# 相当于

deco_fac(1, 2, 3)(f)('a', 'b')    # 不是 deco_fac(1, 2, 3)(f('a', 'b'))，这个只会返回 new_func

a b
a b


## 参数化clock装饰器

**使用参数化装饰器实现clock装饰器中用户提供格式字符串的功能**

In [4]:
import time

DEFAULT_FMT = '[{elapsed:.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):    # 参数化装饰器工厂函数
    def decorate(func):        # 真正的装饰器
        def clocked(*_args):    # 包装被装饰的函数
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))    # locals()为内置函数，返回当前作用域下所有局部变量（名:值）组成的字典
            return _result
        return clocked
    return decorate

In [5]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

In [6]:
for _ in range(3):
    snooze(.123)

[0.12334824s] snooze(0.123) -> None
[0.12379909s] snooze(0.123) -> None
[0.12320852s] snooze(0.123) -> None


In [12]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

In [13]:
for _ in range(3):
    snooze(.123)

snooze: 0.12331271171569824s
snooze: 0.12359380722045898s
snooze: 0.12318968772888184s


In [18]:
@clock('{name}({args}) dt={elapsed:.3f}s')
def snooze(seconds):
    time.sleep(seconds)

In [19]:
for _ in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.124s
